In [1]:
from tqdm import tqdm 
import random
import numpy as np
import copy

In [2]:
def job_time(workload, initial_yield_rate, decrease_rate, lower_bound, pro_rate):  # 計算訂單要花多少時間
    time = 0
    after_rate = initial_yield_rate
    while workload > 0:  # 直到滿足訂單數
        rec_rate = (after_rate/100)*pro_rate
        workload -= rec_rate
        if after_rate > lower_bound:
            after_rate -= decrease_rate
            if after_rate < lower_bound:  # 不能低於良率 lower bound
                after_rate = lower_bound      
        time += 1
    return time, after_rate

In [4]:
def insert_maintenance(time, maintain_t, pro_rate, schedule, tardiness, decrease_rate, L_yield_rate):  # 在某個時間點插入維修重新規劃後的結果
    time += maintain_t  # 加上維修時間
    init_yr = 100  # 修到 100 %
    new_tard = tardiness
    new_schedule = []
    new_tardiness_list = []
    for job_n,t,_ in schedule:  # 剩下還沒有排到的工作
        due = job_dic[job_n][1]
        new_schedule.append((job_n, time, init_yr))
        cost_time, after_yield = job_time(job_dic[job_n][0], init_yr, decrease_rate, L_yield_rate, pro_rate)
        time += cost_time
        init_yr = after_yield
        new_tard += max(time-due,0)  # 只有太晚做完才要算 cost
        new_tardiness_list.append((job_n,new_tard))
    return new_tardiness_list, new_tard, new_schedule

In [5]:
def decide_maintenance(job_schedule, cu_tard, tardiness, h, pro_rate, maintain_time):  # GREEDY 選擇會更好的維修決策
    maintenance_schedule = []  #紀錄修過的紀錄 (機台,時間)
    cnt = 0
    new_cu_tard = copy.deepcopy(cu_tard)
    new_tardiness = copy.deepcopy(tardiness)
    new_job_schedule = copy.deepcopy(job_schedule)
    while cnt < h:  # 最多可以修 h 台
        flag = 0  # 假設該輪沒有找到更好的，就會是 0
        best_improve_tard = 0
        for machine in range(len(job_schedule)):  # 每台機台都跑過
            machine_no = machine + 1
            index = 0  # 紀錄跑到機台上的哪個 job
            m_improve_tard = 0
            m_tard = new_cu_tard[machine_no]  # 初始設定最小 tardiness為原先機台 (machine_no) 的 tardiness
#             print(new_job_schedule)
            for job_no,time,yr in new_job_schedule[machine_no]:  # 每台機台的每個地方插入維修，求最好的
                temp_tardiness_list, temp_tard, temp_schedule = insert_maintenance(time, maintain_time[machine_no], pro_rate[machine_no], new_job_schedule[machine_no][index:], 
                                                            new_tardiness[machine_no][index][1], decrease_rate[machine_no], L_yield_rate[machine_no])  # 計算插入 maintenance 後會不會比較好
                if temp_tard < m_tard:
#                     print("可以改善該機台")
                    flag = 1
                    m_index = index
                    m_improve_tard = m_tard - temp_tard
                    m_tard = temp_tard
                    m_tardiness_list = temp_tardiness_list
                    m_schedule = temp_schedule
                    m_info = (machine_no,time)  # (維修機台, 維修時間)
                index += 1
            if m_improve_tard > best_improve_tard:
#                 print("可以改善整體")
                best_index = m_index
                best_tard = m_tard
                best_tardiness_list = m_tardiness_list
                best_schedule = m_schedule
                best_info = m_info
                best_improve_tard = m_improve_tard
        if flag == 1:  # 維修會更好
            cnt += 1
            new_cu_tard[best_info[0]] = best_tard
            new_tardiness[best_info[0]][best_index:]= best_tardiness_list
            new_job_schedule[best_info[0]][best_index:] = best_schedule
            maintenance_schedule.append(best_info)
        else:
            break
    return maintenance_schedule, new_cu_tard, new_tardiness, new_job_schedule

In [6]:
def find_max_q_job(job_schedule):  # 找出同一層的工作
    pos_jobs = {}
    max_pos = max([len(x) for x in job_schedule.values()])
    position = range(0,max_pos)
    for p in position:
        pos_jobs[p] = []
        for j in job_schedule:
            if len(job_schedule[j]) > p:
                 pos_jobs[p].append(job_schedule[j][p][0])
    return pos_jobs

In [7]:
def reschedule(time, init_yr, new_job, pro_rate, schedule, tardiness, decrease_rate, L_yield_rate):  #將工作重新排列後
    new_tard = tardiness
    new_tardiness_list = []
    new_schedule = []
    # 算新插入的交換工作
    new_schedule.append((new_job, time, init_yr))
    cost_time, after_yield = job_time(job_dic[new_job][0], init_yr, decrease_rate, L_yield_rate, pro_rate)
    time += cost_time
    init_yr = after_yield
    due = job_dic[new_job][1]
    new_tard += max(time-due,0)
    new_tardiness_list.append((new_job,new_tard))
    # 後面依序重新計算
    for job_n,t,_ in schedule:
        due = job_dic[job_n][1]
        new_schedule.append((job_n, time, init_yr))
        cost_time, after_yield = job_time(job_dic[job_n][0], init_yr, decrease_rate, L_yield_rate, pro_rate)
        time += cost_time
        init_yr = after_yield
        new_tard += max(time-due,0)  # 只有太晚做完才要算 cost
        new_tardiness_list.append((job_n,new_tard))
    return new_tardiness_list, new_tard, new_schedule

In [8]:
# 讀入 input
with open("m5n50_3.txt","r") as f:
    m_n_h = [int(x) for x in f.readline().strip().split(' ')]
    m = m_n_h[0]  # 機台數
    n = m_n_h[1]  # 訂單數
    h = m_n_h[2]  # 最多修幾台
    pro_rate = {}  # 生產速率
    decrease_rate = {}  # 機台下降良率
    maintain_time = {}  # 機台維修時間
    initial_yield_rate = {}  # 初始良率
    L_yield_rate = {}  # 最低良率
    for i in range(m):  # 機台編號由 1 開始
        machine_info = [int(x) for x in f.readline().strip().split(' ')]
        pro_rate[i+1] = machine_info[0]
        decrease_rate[i+1] = machine_info[1]
        maintain_time[i+1] = machine_info[2]
        initial_yield_rate[i+1] = machine_info[3]
        L_yield_rate[i+1] = machine_info[4]
    lines = f.readlines()
job_dic = {}  # job_dic[機台編號] = [訂購量,交期]
job_no = 1
for l in lines:
    temp = [int(x) for x in l.strip().split(' ')]
    job_dic[job_no] = temp  
    job_no += 1


In [9]:
# 排工作，不考慮維修 (initail solution)
sort_job_dic = dict(sorted(job_dic.items(), key=lambda item: item[1][0]))
sort_job_dic = dict(sorted(sort_job_dic.items(), key=lambda item: item[1][1]))
# print(sort_job_dic)
machine_workload = {}  # 紀錄結束工作量
job_schedule = {}  #記錄 (job,開始時間,初始良率)
tardiness= {}  #記錄各機台累積的 (job,tardiness)
cu_tard = {}  # 記錄各機台累積 tardiness
init_yr = {}  # 複製紀錄初始良率

In [10]:
# 產生 EDD 排程
for i in range(m):
    machine_workload[i+1] = 0
    job_schedule[i+1] = []
    cu_tard[i+1] = 0
    tardiness[i+1] = []
    init_yr[i+1] = copy.deepcopy(initial_yield_rate[i+1])

for no,[load,due] in tqdm(sort_job_dic.items()):  #利用 EDD + SPT 排完所有工作
    min_machine = sorted(machine_workload.items(), key=lambda item: item[1])[0][0]  # 選目前完成時間最短的機台
    job_schedule[min_machine].append((no,machine_workload[min_machine],init_yr[min_machine]))  
    cost_time, after_yield = job_time(load, init_yr[min_machine], decrease_rate[min_machine], L_yield_rate[min_machine],pro_rate[min_machine])
    machine_workload[min_machine] += cost_time
    init_yr[min_machine] = after_yield
    cu_tard[min_machine] += max(machine_workload[min_machine]-due,0)  # 只有太晚做完才要算 cost
    tardiness[min_machine].append((no,cu_tard[min_machine]))

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 25055.58it/s]


In [12]:
job_schedule

{1: [(23, 0, 90),
  (36, 9, 63),
  (7, 20, 45),
  (45, 36, 45),
  (37, 51, 45),
  (50, 67, 45),
  (49, 81, 45),
  (43, 93, 45),
  (19, 109, 45)],
 2: [(35, 0, 90),
  (1, 10, 60),
  (15, 21, 60),
  (22, 32, 60),
  (33, 41, 60),
  (39, 50, 60),
  (16, 60, 60),
  (13, 70, 60),
  (26, 82, 60),
  (41, 92, 60),
  (42, 103, 60)],
 3: [(18, 0, 90),
  (28, 9, 54),
  (48, 20, 53),
  (44, 30, 53),
  (31, 42, 53),
  (8, 54, 53),
  (40, 64, 53),
  (17, 78, 53),
  (27, 88, 53),
  (25, 100, 53),
  (24, 111, 53)],
 4: [(20, 0, 90),
  (6, 11, 49),
  (14, 25, 49),
  (5, 38, 49),
  (34, 49, 49),
  (32, 60, 49),
  (11, 73, 49),
  (29, 88, 49),
  (30, 100, 49),
  (2, 111, 49)],
 5: [(4, 0, 90),
  (12, 9, 72),
  (9, 19, 52),
  (46, 32, 45),
  (10, 45, 45),
  (47, 60, 45),
  (3, 76, 45),
  (38, 88, 45),
  (21, 102, 45)]}

In [11]:
tardiness

{1: [(23, 0),
  (36, 0),
  (7, 5),
  (45, 23),
  (37, 55),
  (50, 99),
  (49, 154),
  (43, 224),
  (19, 303)],
 2: [(35, 0),
  (1, 0),
  (15, 0),
  (22, 8),
  (33, 24),
  (39, 49),
  (16, 83),
  (13, 128),
  (26, 182),
  (41, 246),
  (42, 320)],
 3: [(18, 0),
  (28, 0),
  (48, 0),
  (44, 10),
  (31, 30),
  (8, 58),
  (40, 100),
  (17, 150),
  (27, 212),
  (25, 283),
  (24, 363)],
 4: [(20, 0),
  (6, 0),
  (14, 6),
  (5, 21),
  (34, 46),
  (32, 83),
  (11, 134),
  (29, 195),
  (30, 265),
  (2, 347)],
 5: [(4, 0),
  (12, 0),
  (9, 1),
  (46, 13),
  (10, 39),
  (47, 79),
  (3, 129),
  (38, 192),
  (21, 266)]}

In [253]:
print('工作排程：\n',job_schedule)
print('機台累積 tardiness：\n',cu_tard)
print('總 tardiness：\n',sum(cu_tard.values()))

工作排程：
 {1: [(23, 0, 90), (36, 9, 63), (7, 20, 45), (45, 36, 45), (37, 51, 45), (50, 67, 45), (49, 81, 45), (43, 93, 45), (19, 109, 45)], 2: [(35, 0, 90), (1, 10, 60), (15, 21, 60), (22, 32, 60), (33, 41, 60), (39, 50, 60), (16, 60, 60), (13, 70, 60), (26, 82, 60), (41, 92, 60), (42, 103, 60)], 3: [(18, 0, 90), (28, 9, 54), (48, 20, 53), (44, 30, 53), (31, 42, 53), (8, 54, 53), (40, 64, 53), (17, 78, 53), (27, 88, 53), (25, 100, 53), (24, 111, 53)], 4: [(20, 0, 90), (6, 11, 49), (14, 25, 49), (5, 38, 49), (34, 49, 49), (32, 60, 49), (11, 73, 49), (29, 88, 49), (30, 100, 49), (2, 111, 49)], 5: [(4, 0, 90), (12, 9, 72), (9, 19, 52), (46, 32, 45), (10, 45, 45), (47, 60, 45), (3, 76, 45), (38, 88, 45), (21, 102, 45)]}
機台累積 tardiness：
 {1: 303, 2: 320, 3: 363, 4: 347, 5: 266}
總 tardiness：
 1599


In [254]:
# 初始解 GREEDY 找維修
maint_sche, new_cu_tard, new_tardiness, new_job_schedule = decide_maintenance(job_schedule, cu_tard, tardiness, h, pro_rate, maintain_time)

In [255]:
print('工作排程：\n',new_job_schedule)
print('維修時程：\n',maint_sche)
print('機台累積 tardiness：\n',new_cu_tard)
print('總 tardiness：\n',sum(new_cu_tard.values()))

工作排程：
 {1: [(23, 0, 90), (36, 9, 63), (7, 20, 45), (45, 36, 45), (37, 51, 45), (50, 67, 45), (49, 81, 45), (43, 93, 45), (19, 109, 45)], 2: [(35, 0, 90), (1, 10, 60), (15, 21, 60), (22, 32, 60), (33, 41, 60), (39, 50, 60), (16, 60, 60), (13, 70, 60), (26, 82, 60), (41, 92, 60), (42, 103, 60)], 3: [(18, 0, 90), (28, 11, 100), (48, 19, 100), (44, 25, 76), (31, 36, 53), (8, 48, 53), (40, 58, 53), (17, 72, 53), (27, 82, 53), (25, 94, 53), (24, 105, 53)], 4: [(20, 0, 90), (6, 13, 100), (14, 24, 100), (5, 34, 100), (34, 41, 58), (32, 52, 49), (11, 65, 49), (29, 80, 49), (30, 92, 49), (2, 103, 49)], 5: [(4, 0, 90), (12, 9, 72), (9, 19, 52), (46, 32, 45), (10, 45, 45), (47, 60, 45), (3, 76, 45), (38, 88, 45), (21, 102, 45)]}
維修時程：
 [(3, 9), (4, 11), (4, 22), (3, 17), (4, 32)]
機台累積 tardiness：
 {1: 303, 2: 320, 3: 315, 4: 297, 5: 266}
總 tardiness：
 1501


In [256]:
base_maint_tard = sum(new_cu_tard.values())  # 紀錄使用 EDD + GREEDY 解出的 tardiness 作為 benchmark
base_maint_tard

1501

In [257]:
# Tabu 
# def Tabu(max_tabu_size, base_maint_tard, job_schedule, pro_rate, tardiness, decrease_rate, L_yield_rate, cu_tard, swap_maint_sche):

max_tabu_size = 30  # 避免重覆交換的 list
tabu_list = []
swap_job_schedule = copy.deepcopy(job_schedule)
swap_cu_tard = copy.deepcopy(cu_tard)
swap_tardiness = copy.deepcopy(tardiness)
swap_maint_sche = copy.deepcopy(maint_sche)

# 將工作分層
job_level = find_max_q_job(job_schedule)
no_better = 0
while no_better < 50:  # 連續 50 次交換都沒有更好，則停止
    # 選層
    rand_j = random.randint(0, len(job_level)-1)
    while len(job_level[rand_j]) < 2:
        rand_j = random.randint(0, len(job_level)-1)
    # 選 job
    resultList = random.sample(range(0, len(job_level[rand_j])),2)
    job_1 = job_level[rand_j][resultList[0]]
    job_2 = job_level[rand_j][resultList[1]]
    if (job_1,job_2) or (job_2,job_1) not in tabu_list: # 沒有換過，才要換換看
#         print(job_1,job_2)
        # 先建立一個紀錄交換排程用的資料
        temp_job_schedule = copy.deepcopy(swap_job_schedule)
        temp_cu_tard = copy.deepcopy(swap_cu_tard)
        temp_tardiness = copy.deepcopy(swap_tardiness)
        for i in range(m):  # 找出要交換的工作機台時間位址並重新規劃
            machine = i+1
            index = 0
            for job_n, t, init_yr in job_schedule[machine]:
                if job_n == job_1:
                    machine_1 = machine 
                    index_1 = index 
                    new_tardiness_list_1, new_tard_1, new_schedule_1 = reschedule(t, init_yr, job_2, pro_rate[machine], job_schedule[machine][index+1:],
                                                                            tardiness[machine][index][1], decrease_rate[machine], L_yield_rate[machine])

                elif job_n == job_2:
                    machine_2 = machine 
                    index_2 = index 
                    new_tardiness_list_2, new_tard_2, new_schedule_2 = reschedule(t, init_yr, job_1, pro_rate[machine], job_schedule[machine][index+1:],
                                                                            tardiness[machine][index][1], decrease_rate[machine], L_yield_rate[machine])
                index += 1
        # 將交換排程紀錄下來
        temp_tardiness[machine_1][index_1:] = new_tardiness_list_1 
        temp_tardiness[machine_2][index_2:] = new_tardiness_list_2
        temp_cu_tard[machine_1] = new_tard_1
        temp_cu_tard[machine_2] = new_tard_2
        temp_job_schedule[machine_1][index_1:] = new_schedule_1 
        temp_job_schedule[machine_2][index_2:] = new_schedule_2 
        
        # 交換後，插入維修
        temp_maint_sche, temp_new_cu_tard, temp_new_tardiness, temp_new_job_schedule = decide_maintenance(temp_job_schedule, temp_cu_tard, temp_tardiness, h, pro_rate, maintain_time)
        if  sum(temp_new_cu_tard.values()) < base_maint_tard:  # 表示新的交換有進步，則更新排程
            print("improve! tardiness = ",sum(temp_new_cu_tard.values()))
            no_better = 0  # 將連續找不到的次數歸零
            # 更新排程資訊
            base_maint_tard = sum(temp_new_cu_tard.values())
            swap_tardiness = temp_tardiness
            swap_cu_tard = temp_cu_tard
            swap_job_schedule = temp_job_schedule
        else:
             no_better += 1  # 將連續找不到的次數 + 1
        # 沒看過，更新 tabu list 資訊
        tabu_list.append((job_1,job_2))
        if len(tabu_list) > max_tabu_size:
            tabu_list.pop(0)
    else:
        no_better += 1  # 將連續找不到的次數 + 1           

improve! tardiness =  1445
improve! tardiness =  1422
improve! tardiness =  1417
improve! tardiness =  1408
improve! tardiness =  1369
improve! tardiness =  1365
improve! tardiness =  1359
improve! tardiness =  1357
improve! tardiness =  1349
improve! tardiness =  1340
improve! tardiness =  1335
improve! tardiness =  1334


In [154]:
# 把最好的排程找維修決策
swap_maint_sche, swap_maint_cu_tard, swap_maint_tardiness, swap_maint_job_schedule = decide_maintenance(swap_job_schedule, swap_cu_tard, swap_tardiness, h, pro_rate,maintain_time)

In [155]:
# print('工作排程：\n',swap_maint_job_schedule)
print('維修時程：\n',swap_maint_sche)
# print('機台累積 tardiness：\n',swap_maint_cu_tard)
print('總 tardiness：\n',sum(swap_maint_cu_tard.values()))

維修時程：
 [(1, 0), (1, 163)]
總 tardiness：
 503793


In [145]:
# 找要維修的分多一點工作 e.g.找每層最大訂單數給他，並且至少補到平均層數
# num_list = []
# for j in job_schedule:
#     num_list.append(len(job_schedule[j]))
# mean_num = int(np.mean(num_list))
# for m,t in maint_sche:
#     if len(job_schedule[m]) < mean_num:
#         i = num_list.index(max(num_list))
#         swap(job_schedule[i][-1],job_schedule[m])

In [10]:
import itertools
aa = [1,2,3]
k = [(0,0)]
k.extend(list(itertools.combinations(aa, 2)))
k

[(0, 0), (1, 2), (1, 3), (2, 3)]

In [20]:
def new_simple(job1, job2, simple_sche):
    new_sche = copy.deepcopy(simple_sche)
    for m in simple_sche.keys():
        for i in range(len(simple_sche[m])):
            if simple_sche[m][i] == job1:
                new_sche[m][i] = job2
            elif simple_sche[m][i] == job2:
                new_sche[m][i] = job1 
    return new_sche

In [21]:
simple_sche = {1: [6, 5, 4, 10, 8], 2: [1, 7, 3, 9, 2]}

In [22]:
job_1 = 6
job_2 = 1
new_simple_shce =  new_simple(job_1, job_2, simple_sche)

In [23]:
new_simple_shce

{1: [1, 5, 4, 10, 8], 2: [6, 7, 3, 9, 2]}

In [2]:
from functions import *
from tqdm import tqdm 
def EDD_alt(m, n, h, pro_rate, decrease_rate, maintain_time, initial_yield_rate, L_yield_rate, job_dic):

    # 排工作，不考慮維修 (initail solution)
    sort_job_dic = dict(sorted(job_dic.items(), key=lambda item: item[1][0]))
    sort_job_dic = dict(sorted(sort_job_dic.items(), key=lambda item: item[1][1]))

    machine_workload = {}  # 紀錄結束工作量
    job_schedule = {}  #記錄 (job,開始時間,初始良率)
    loss= {}  #記錄各機台累積的 (job,tardiness)
    cu_loss = {}  # 記錄各機台累積 tardiness
    init_yr = {}  # 複製紀錄初始良率
    
    # 產生 EDD 排程
    for i in range(m):
        machine_workload[i+1] = 0
        job_schedule[i+1] = []
        cu_loss[i+1] = 0
        loss[i+1] = []
        init_yr[i+1] = initial_yield_rate[i+1]
    print("正在用 EDD rule 產生排程：")
    for no,[load,due] in tqdm(sort_job_dic.items()):  # 利用 EDD + SPT 排完所有工作
        min_machine = select_machine(machine_workload)  # 選目前完成時間最短的機台
        job_schedule[min_machine].append((no,machine_workload[min_machine],init_yr[min_machine]))
        after_rate = init_yr[min_machine]
        temp_load = load
        while temp_load > 0 and machine_workload[min_machine] < due:  # 未完成訂單且未到 due day
            rec_rate = (after_rate/100)*pro_rate
            temp_load -= rec_rate
            if after_rate > L_yield_rate:
                after_rate -= decrease_rate
                if after_rate < L_yield_rate:  # 不能低於良率 lower bound
                    after_rate = L_yield_rate
            machine_workload[min_machine] += 1
        init_yr[min_machine] = after_rate
        if temp_load > 0:  # 未滿足交期
            cu_loss[min_machine] += temp_load  # 只有太晚做完才要算 cost
        loss[min_machine].append((no,cu_loss[min_machine]))
    print('維修前的 EDD 排程總 tardiness：\n',sum(cu_loss.values()))
    
    # 初始解 GREEDY 找維修
    maint_sche, new_cu_tard, new_tardiness, new_job_schedule = \
        alt_decide_maintenance(decrease_rate, job_schedule, cu_loss, loss, h, pro_rate, maintain_time, job_dic, L_yield_rate)
    
    print('產生維修時程：\n',maint_sche)
    print('維修後的 EDD 排程總 production loss = ',sum(new_cu_tard.values()))
    edd_obj = sum(new_cu_tard.values())
    
    return job_schedule, tardiness, cu_tard, maint_sche, new_cu_tard, new_tardiness, new_job_schedule, edd_obj

2